# Geographical Plotting

### Instructor: Rossano Schifanella
Email: [rossano.schifanella@unito.it](mailto:rossano.schifanella@unito.it)

# Mapping in Python with geopandas

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import geopandas as gpd
import pysal as ps
import palettable as pltt
import seaborn as sns
from seaborn import palplot
import numpy as np

Environment:

In [ ]:
print('matplotlib ', matplotlib.__version__)
print('geopandas ',gpd.__version__)
print('pysal ',ps.__version__)

## Loading up spatial data 

The easiest way to get from a file to a quick visualization of the data is by loading it as a `GeoDataFrame` and calling the `plot` command. The main library employed for all of this is `geopandas` which is a geospatial extension of the `pandas` library, already introduced before. `geopandas` supports exactly the same functionality that `pandas` does (in fact since it is built on top of it, so most of the underlying machinery is pure `pandas`), plus a wide range of spatial counterparts that make manipulation and general "munging" of spatial data as easy as non-spatial tables.

In two lines of code, we will obtain a graphical representation of the spatial data contained in a file that can be in many formats; actually, since it uses the same drivers under the hood, you can load pretty much the same kind of vector files that QGIS permits. Let us start by plotting single layers in a crude but quick form, and we will build style and sophistication into our plots later on.

Refer to this [list](https://en.wikipedia.org/wiki/GIS_file_formats#Vector) for a description of the most common vectorial file formats. 

#### Polygons

Let us begin with the most common type of spatial data in the social science: polygons. For example, we can load the geography of LSOAs in London with the following lines of code:

In [ ]:
london_msoa = gpd.read_file('data/shp/london_msoa/msoa.shp')
london_msoa.columns.values

In [ ]:
london_msoa.plot()

This might not be the most aesthetically pleasant visual representation of the MSOA geography, but it is hard to argue it is not quick to produce. We will work on styling and customizing spatial plots later on.

If you call a single row of the `geometry` column, it'll return a small plot of the shape:

In [ ]:
london_msoa.loc[0, 'geometry']

#### Lines
Displaying lines is as straight-forward as polygons. To load tube lines in London:

In [ ]:
london_tube = gpd.read_file('data/shp/london_tublines.shp')
london_tube.head(2)

In [ ]:
london_tube.plot()

#### Points
Finally, points follow a similar structure. If we want to represent named places in London:

In [ ]:
london_pois = gpd.read_file('data/shp/gis.osm_pois_free_1.shp')

In [ ]:
london_pois.head(2)

In [ ]:
london_pois.plot(edgecolor='black')

Note that the coordinates of the point of interests dataset are in a different projection (EPSG:4326) that the lines and polygons geometries (EPSG:27700). Geopandas provides the function `to_crs` that is able to covert the Coordinate Reference System (CRS) of a geometry specifying the target projection. This is useful in our use case to visualize multiple spatial datasets in a single plot (they need to have the same coordinates system). 

Refer to [https://geopandas.org/projections.html](https://geopandas.org/projections.html) for a description on how `geopandas` manages projections. 

Projections can be identified using the [EPSG](http://www.epsg.org/) standard. The IOGP’s EPSG Geodetic Parameter Dataset is a collection of definitions of coordinate reference systems and coordinate transformations which may be global, regional, national or local in application. 

In [ ]:
london_pois = london_pois.to_crs("EPSG:27700")

In [ ]:
london_pois.head(2)

You can save a spatial dataframe with the method `to_file`

In [ ]:
london_pois.to_file(driver = 'ESRI Shapefile', filename= "data/shp/pois_london_epsg27700.shp")

List of the types of point of interets in the dataset

In [ ]:
london_pois['fclass'].unique()

Plot the pubs in London

In [ ]:
london_pubs = london_pois.loc[london_pois['fclass']=="pub"]
london_pubs.info()

In [ ]:
london_pubs.plot(edgecolor='black')

## Styling plots

It is possible to tweak several aspects of a plot to customize if to particular needs. In this section, we will explore some of the basic elements that will allow us to obtain more compelling maps.

* Remove axis
* Set title
* Customizing colors, lines, alpha
* Customizing size

These are styling options. Omit them or alter them to suit.

In [ ]:
from matplotlib import rc

rc('font', **{'family':'DejaVu Sans',
    'sans-serif':['Helvetica'],
    'monospace': ['Inconsolata'],
    'serif': ['Adobe Garamond Pro']}
  )

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# Plot layer of polygons on the axis
london_msoa.plot(
    ax=ax, 
    linewidth=0.3, 
    facecolor='#999999', 
    edgecolor='#111111', 
    alpha=0.8)

# Display
plt.title('Greater London LSOAs')
plt.tight_layout()
plt.show()    

## Composing multi-layer maps

So far we have considered many aspects of plotting a single layer of data. However, in many cases, an effective map will require more than one: for example we might want to display streets on top of the polygons of neighborhoods, and add a few points for specific locations we want to highlight. At the very heart of GIS is the possibility to combine spatial information from different sources by overlaying it on top of each other, and this is fully supported in Python.

Essentially, combining different layers on a single map boils down to adding each of them to the same axis in a sequential way, as if we were literally overlaying one on top of the previous one. For example, let us get the simplest possible plot, one with the polygons from the LSOAs and the tube lines on top of them:

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(10., 8.),
    dpi=100,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# Plot layer of polygons on the axis
london_msoa.plot(ax=ax, linewidth=0.2, facecolor='#999999', edgecolor='#111111', alpha=0.8)
london_tube.plot(ax=ax, linewidth=1, edgecolor='red', alpha=0.6, label='tube lines')
london_pubs.plot(ax=ax, linewidth=0.6, facecolor='blue', alpha=0.8, markersize=2, label='pubs')

# Display
ax.legend()
plt.title('Greater London LSOAs+tube+pubs')
plt.tight_layout()

## Saving maps to figures 

Once we have produced a map we are content with, we might want to save it to a file so we can include it into a report, article, website, etc. Exporting maps in Python is as simple as using the method `plt.savefig` at the end of the code block to specify where and how to save it. For example to save the previous map into a `png` file in the same folder where the notebook is hosted:

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(10., 8.),
    dpi=100,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# Plot layer of polygons on the axis
london_msoa.plot(ax=ax, linewidth=0.2, facecolor='#999999', edgecolor='#111111', alpha=0.8)
london_tube.plot(ax=ax, linewidth=1, edgecolor='red', alpha=0.6, label='tube lines')
london_pubs.plot(ax=ax, linewidth=0.6, facecolor='blue', alpha=0.8, markersize=2, label='pubs')

ax.legend()

plt.title('Greater London LSOAs+tube+pubs')
plt.savefig('figs/london_tube_pubs.png', dpi=300, transparent=False, tight_layout=True)

Loading the London [Index of Multiple Deprivation](https://data.london.gov.uk/dataset/indices-of-deprivation) (IMD) dataset 

In [ ]:
imd_london = gpd.read_file('data/shp/London_IMD_MSOA.shp')
imd_london.head(2)

In [ ]:
imd_london.columns.values

# Choropleths

## Unique values

A choropleth for categorical variables simply assigns a different color to every potential value in the series. The main requirement in this case is then for the color scheme to reflect the fact that different values are not ordered or follow a particular scale.

In Python, thanks to `geopandas`, creating categorical choropleths is possible with one line of code. To demonstrate this, we can plot the spatial distribution of LSOAs with a more white population than other races and viceversa:

In [ ]:
imd_london['majority_race'] = np.where(imd_london.eval("white_p > 50"), "White", "Other")

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=150,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')

lines = imd_london.plot(
    ax=ax,
    column='majority_race',
    categorical=True,
    alpha=1, 
    edgecolor='w', 
    linewidth=0.3, 
    legend=True, 
    antialiased=True
)

plt.tight_layout()
plt.show()


Let us stop for a second in a few crucial aspects:

* Note how we are using the same approach as for basic maps, the command `plot`, but we now need to add the argument `column` to specify which column in particular is to be represented.
* Since the variable is categorical we need to make that explicit by setting the argument `categorical` to `True`.
* As an optional argument, we can set `legend` to `True` and the resulting figure will include a legend with the names of all the values in the map.
* Unless we specify a different colormap, the selected one respects the categorical nature of the data by not implying a gradient or scale but a qualitative structure.

## Equal intervals

If, instead of categorical variables, we want to display the geographical distribution of a continuous phenomenon, we need to select a way to encode each value into a color. One potential solution is applying what is usually called "equal intervals". The intuition of this method is to split the *range* of the distribution, the difference between the minimum and maximum value, into equally large segments and to assign a different color to each of them according to a palette that reflects the fact that values are ordered.

Using the example of the position of a LSOA in the national ranking of the IMD (`imd_rank`), we can calculate these segments, also called bins or buckets, using the library `mapclassify`:

In [ ]:
imd_attribute = 'crime'

In [ ]:
from mapclassify import EqualInterval
from mapclassify import Quantiles
from mapclassify import FisherJenks

In [ ]:
classes = EqualInterval(imd_london[imd_attribute], k=7)
classes

The only additional argument to pass to `EqualInterval`, other than the actual variable we would like to classify is the number of segments we want to create, `k`, which we are arbitrarily setting to seven in this case. This will be the number of colors that will be plotted on the map so, although having several can give more detail, at some point the marginal value of an additional one is fairly limited, given the ability of the brain to tell any differences.

Once we have classified the variable, we can check the actual break points where values stop being in one class and become part of the next one:

In [ ]:
classes.bins

In [ ]:
def draw_bins(geodf, attribute, scheme):

    schemes = {'equal_interval': EqualInterval, \
               'quantiles': Quantiles, \
               'fisher_jenks': FisherJenks}
    
    classi = schemes[scheme](geodf[attribute], k=7)
    
    # Set up the figure
    f, ax = plt.subplots(1)
    # Plot the kernel density estimation (KDE)
    sns.kdeplot(geodf[attribute], shade=True)
    # Add a blue tick for every value at the bottom of the plot (rugs)
    sns.rugplot(geodf[attribute], alpha=0.5)
    # Loop over each break point and plot a vertical red line
    for cut in classi.bins:
        plt.axvline(cut, color='red', linewidth=0.75)
    # Display image
    plt.show()

draw_bins(imd_london, imd_attribute, 'equal_interval')

Technically speaking, the figure is created by overlaying a KDE plot with vertical bars for each of the break points. This makes much more explicit the issue highlighed by which the first bin contains a large amount of observations while the one with top values only encompasses a handful of them.

To create a map that displays the colors assigned by the equal interval classification algorithm, we use a similar approach as with unique values but with some key differences:

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    subplot_kw=dict(aspect='equal'))

ax.axis('off')
ax.set_title("London IMD 2015")

imd_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='equal_interval', k=7, 
    cmap=plt.cm.Blues_r, 
    alpha=1, 
    edgecolor='w', 
    linewidth=0.1, 
    legend=True, 
    antialiased=True
)

plt.tight_layout()
plt.show()

We can use the parameter `legend_kwds` to customize the legend appearance:

In [ ]:
legend_kwds_default = {    
    'loc':'lower right', 
    'fontsize':10, 
    'shadow':True, 
    'fancybox':True, 
    'frameon':True,
    'framealpha':1.0,
    'edgecolor':'#999999',
    'facecolor':'#FFFFFF',
    'bbox_to_anchor':(1, 0.1)
}

def draw_choropleth(geodf, attribute, scheme, cmap=plt.cm.Blues_r, dpi=200, width=12., 
                    height=8., title=None,legend_kwds=legend_kwds_default, smallprint=None):
    f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    subplot_kw=dict(aspect='equal'))

    ax.axis('off')
    
    if title:
        ax.set_title(title)

    geodf.plot(
        ax=ax,
        column=attribute, 
        scheme=scheme, 
        k=7, 
        cmap=cmap, 
        alpha=1, 
        edgecolor='w', 
        linewidth=0.1, 
        legend=True, 
        antialiased=True,
        legend_kwds=legend_kwds_default
    )

    # Display data sources
    if smallprint:
        ax.text(smallprint,
            ha='right', va='bottom',
            size=10,
            color='#555555',
            transform=ax.transAxes)

    plt.tight_layout()
    plt.show()
    
    
    

## Quantiles 

One solution to obtain a more balanced classification scheme is using quantiles. This, by definition, assigns the same amount of values to each bin: the entire series is laid out in order and break points are assigned in a way that leaves exactly the same amount of observations between each of them. This "observation-based" approach contrasts with the "value-based" method of equal intervals and, although it can obscure the magnitude of extreme values, it can be more informative in cases with skewed distributions.

Calculating a quantiles classification with `PySAL` can be done with the following line of code:

In [ ]:
classes = Quantiles(imd_london[imd_attribute], k=7)
classes

And, similarly, the bins can also be inspected:

In [ ]:
classes.bins

The visualization of the distribution can be generated in a similar way as well:

In [ ]:
draw_bins(imd_london, imd_attribute, 'quantiles')

And the choropleth also follows a similar pattern, with the difference that we are now using the scheme "quantiles", instead of "equal interval":

In [ ]:
draw_choropleth(imd_london, imd_attribute, 'quantiles', title="London IMD 2015")

## Fisher-Jenks

Equal interval and quantiles are only two examples of very many classification schemes to encode values into colors. Although not all of them are integrated into `geopandas`, `PySAL` includes several other classification schemes (for a detailed list, have a look at this [link](http://pysal.readthedocs.org/en/latest/library/esda/mapclassify.html)). As an example of a more sophisticated one, let us create a Fisher-Jenks choropleth:

In [ ]:
classes = FisherJenks(imd_london[imd_attribute], k=7)
classes

In [ ]:
classes.bins

In [ ]:
draw_bins(imd_london, imd_attribute, 'fisher_jenks')

Technically, however, the way to create a Fisher-Jenks map is exactly the same as before (it takes time!):

In [ ]:
draw_choropleth(imd_london, imd_attribute, 'fisher_jenks', title="London IMD 2015")

## Zooming into the map (trivial)

A general map of an entire region, or urban area, can sometimes obscure particularly local patterns because they happen at a much smaller scale that cannot be perceived in the global view. One way to solve this is by providing a focus of a smaller part of the map in a separate figure. Although there are many ways to do this in Python, the most straightforward one is to reset the limits of the axes to center them in the area of interest.

As an example, let us consider the quantile map produced above:

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(4., 3.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')

imd_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='quantiles', k=7, 
    cmap=plt.cm.Blues_r, 
    edgecolor='black',
    lw=0.05,
    alpha=1.0
)

# Redimensionate X and Y axes to desired bounds
ax.set_ylim(175000, 185000)
ax.set_xlim(530000, 540000)
plt.tight_layout()
plt.show()


## Zooming into the map (advanced)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import ListedColormap

# Create a custom colormap
n_classes = 5
vals = np.ones((n_classes, 4))
vals[:, 0]=vals[:, 1]=vals[:, 2]=np.linspace(0.3, 0.8, n_classes)
newcmp = ListedColormap(vals)

# Load London boroughs
boroughs_df = gpd.read_file("data/shp/London_Borough_Excluding_MHW.shp")
# Select the subset of boroughs of interest
boroughs_selected = ['E09000007', 'E09000033','E09000019','E09000020','E09000001']
boroughs_df = boroughs_df.loc[boroughs_df['GSS_CODE'].isin(boroughs_selected)]


# Create matplot figure and axes
fig, ax = plt.subplots(1, figsize=(12., 10.), subplot_kw=dict(aspect='equal'))

ax.axis('off')
ax.set_title("Crime rate")

scheme = 'Quantiles'

# Plot the choropleth selecting a subset of bouroughs 
imd_london.loc[imd_london['lad11cd'].isin(boroughs_selected)].plot(
    ax=ax,
    alpha=0.8,
    column=imd_attribute,
    k=n_classes,
    cmap=newcmp,
    linewidth=0.1,
    edgecolor='black',
    legend=True, 
    legend_kwds = legend_kwds_default,
    scheme=scheme,
)

# Plot the boundaries of the boroughs (thick blue lines)
boroughs_df.plot(ax=ax, 
                 lw=4, 
                 color="none", 
                 edgecolor="#007bba", alpha=0.7)


# Create the internal plot inset
axins = inset_axes(
    ax,
    width="20%", # width = % of parent_bbox
    height=2.0,
    loc=2,
    axes_kwargs={'aspect': 'equal', 'facecolor': '#555555'}
)
axins.axis('off')

# Plot the London shape in the internal inset
inset= imd_london.plot(
    ax=axins,
    color='#007bba',
    facecolor='#007bba',
    edgecolor='#007bba',
    legend=False,
)

# Plot the contour of the selected bourough (white thin line)
union = boroughs_df.unary_union
union_gdf = gpd.GeoDataFrame(geometry=[union])
union_gdf.plot(ax=axins, linewidth=1, edgecolor='white', color='none')

# Add textual description 
smallprint = ax.text(
    .995, 0,
    'Classification Method: %s\n \
    Contains National Statistics data\n \
    Contains Ordnance Survey data on %d LSOAs\n' % (scheme, imd_london['lad11cd'].isin(boroughs_selected).sum()),
    ha='right', va='bottom',
    size=10,
    color='#555555',
    transform=ax.transAxes)


plt.show()